# Basic Proxy Signature Protocol for Partial Delegation

## Greatest Common Divisor

We need this function in the signature scheme as a utility function.

In [29]:
def gcd(a, b):
    if((a<0) or (b<0) or (a<b)):
        print("wrong parameter input")
        return

    while(b != 0):
        r = a % b
        a = b
        b = r
        
    return a

In [30]:
a = 31
b = 23
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

a = 16
b = 9
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

a = 468
b = 249
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

gcd( 31 , 23 )= 1
gcd( 16 , 9 )= 1
gcd( 468 , 249 )= 3


# Protocol

## Setup: Key pair generation of original signer

public parameter - modulus: prime $p$

public parameter - generator for $Z^*_p$: $g$

public key: v

secret key: s

__In this test implementation, only input needs to be given by the user is the prime modulus $p$.__

In [31]:
import random

# Miller-Rabin Primality Test
def miller_rabin(n, k=5):  # number of rounds k=5 gives a good balance between accuracy and performance
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False
    
    # Write n-1 as 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2
    
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

# Function to generate a large prime number
def generate_large_prime(bits=32):
    while True:
        p = random.getrandbits(bits)
        if p % 2 != 0 and miller_rabin(p):
            return p

# Function to find a generator for Z_p
def find_generator(p):
    # Find factors of p-1
    factors = []
    phi = p - 1
    n = phi
    i = 2
    while i * i <= n:
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
        i += 1
    if n > 1:
        factors.append(n)
    
    for g in range(2, p):
        valid_generator = True
        for factor in factors:
            if pow(g, phi // factor, p) == 1:
                valid_generator = False
                break
        if valid_generator:
            return g
    return None

# Generate a large prime number
p = generate_large_prime(64)

# Find a generator g for Z_p
g = find_generator(p)

# Generate secret key
s = random.randint(1, p-2)

# Compute public key
v = pow(g, s, p)

print('*public parameters*')
print('prime modulus (p):', p)
print('generator     (g):', g)
print('*key pair of original signer*')
print('public key    (v):', v)
print('secret key    (s):', s)


*public parameters*
prime modulus (p): 5032252537865353033
generator     (g): 5
*key pair of original signer*
public key    (v): 2012982231122774110
secret key    (s): 2614340758240045521


## Step 1: Proxy generation

Proxy: ($\sigma$, K)

In [32]:
k = random.randint(1, p-2)  # Z_{p-1}/{0}


print('(just any) random number k:', k)

print('*proxy value pair for proxy signer*')

# Efficiently compute K = g^k % p using the pow function
bigk = pow(g, k, p)
print('K    :', bigk)

sigma = (s + k * bigk) % (p - 1)
print('sigma:', sigma)


(just any) random number k: 4641269063811794972
*proxy value pair for proxy signer*
K    : 1703719962413632553
sigma: 3377736686810704213


## Step 2: Proxy delivery

The proxy $(\sigma,K)$, must be given by original signer to proxy signer securely.

For example, using Diffie-Hellman to create a secure tunnel, this can be done.

## Step 3: Proxy verification

In [33]:
# Efficiently compute LHS = g^sigma % p
lhs = pow(g, sigma, p)

# Efficiently compute RHS = (v * bigk^bigk) % p
# First, calculate bigk^bigk % p, then multiply by v, and finally take mod p
rhs = (v * pow(bigk, bigk, p)) % p

print('LHS:', lhs)
print('RHS:', rhs)

if lhs == rhs:
    print('Proxy Verification: Passed')
else:
    print('Proxy Verification: Failed')


LHS: 1706151432803013004
RHS: 1706151432803013004
Proxy Verification: Passed


## Step 4: Signing by the proxy signer

We are going to use __ElGamal signature scheme__.

The __ElGamal signing__ protocol to sign message $m$ where the secret key of the signer is $x$:
1. Choose an integer $k$ randomly from $\{ 2 \ldots p−2 \}$  with $k$ relatively prime to $p−1$.
2. Compute $r \equiv g^{k} \pmod p$.
3. Compute $s \equiv (H(m)-xr)k^{-1} \pmod {p-1}$.
If s = 0, then start again with a different random k.

The signature is $(r,s)$. 

When generating the proxy signature, use $\sigma$ instead of the secret key $x$.

In [34]:
import hashlib
# Message to sign, treated as H(m), where m is randomly chosen in the range [1, p-1]
# m = random.randint(1, p-1)
# m = 191961389165164313261613516115364513513189135162935136113513615613

# get the message as input from a message.txt file
def read_message_from_file(file_name):
    with open(file_name, 'r') as file:
        # Read the message from the file
        message = file.read()
        print('message:', message)
        # Convert the message to a sha256 hash
        sha256_hash = hashlib.sha256(message.encode()).hexdigest()
        
    return int(sha256_hash, 16) # Convert the hash to an integer
    
message_filename = 'message.txt'
m = read_message_from_file(message_filename)

print('message (m):', m)

# Find k such that gcd(k, p-1) = 1
k = 0
for i in range(1000):  # Maximum 1000 attempts to find a suitable k
    k = random.randint(2, p-2)
    if gcd(p-1, k) == 1:
        break

print('(a special) random number k:', k, '(found after', i, 'tries. This is relatively prime to p-1 =', p-1, ')')

# Compute r ≡ g^k mod p using pow function for efficiency
r = pow(g, k, p)
print('r    :', r)

s1 = (m - (sigma * r))

# The signature is (r, s1)
signature = (r, s1)
print('Signature:', signature)

message: Ado mama Anjula
message (m): 92936444892425296126897496249657084158512872626459328642006554000005634312468
(a special) random number k: 3749014329375830993 (found after 2 tries. This is relatively prime to p-1 = 5032252537865353032 )
r    : 2957268667464979456
Signature: (2957268667464979456, 92936444892425296126897496249657084158502883751588476376110099722314696664340)


In [35]:
# Calculate the modular inverse of k mod q efficiently

def extended_gcd(k, q):
    if k == 0:
        return q, 0, 1
    gcd, x1, y1 = extended_gcd(q % k, k)
    x = y1 - (q // k) * x1
    y = x1
    return gcd, x, y

def mod_inverse(k, q):
    gcd, invk, _ = extended_gcd(k, q)
    if gcd != 1:
        raise ValueError(f"Inverse doesn't exist for k = {k} mod q = {q}, as they are not coprime.")
    return invk % q

q = p-1
invk = mod_inverse(k, q)
print('Modulo inverse of k mod q:', invk, '(for', k, 'mod', q, ')')


Modulo inverse of k mod q: 4520124239856133097 (for 3749014329375830993 mod 5032252537865353032 )


In [36]:
# Compute the final signature component s
s = (s1 * invk) % (p - 1)
print('s:', s)

# Output the final proxy-signed message
print('The proxy signed message is (m, (r, s), K): (', m, '(', r, ',', s, ')', bigk, ')')

# Function to write a proxy signature to a file
def write_signature_to_file(filename, signature):
    with open(filename, 'w') as file:
        file.write(f"r: {signature[0]}\n")
        file.write(f"s: {signature[1]}\n")
        file.write(f"K: {signature[2]}\n")

# Write the proxy signature to a proxy_signature.txt
signature_filename = 'proxy_signature.txt'
write_signature_to_file(signature_filename, (r, s, bigk))
print(f"Proxy signature written to file: {signature_filename}")


s: 1135264699249436940
The proxy signed message is (m, (r, s), K): ( 92936444892425296126897496249657084158512872626459328642006554000005634312468 ( 2957268667464979456 , 1135264699249436940 ) 1703719962413632553 )
Proxy signature written to file: proxy_signature.txt


## Step 5: Verification of the proxy signature

The new public key $v' \equiv vK^K \pmod p$

The __ElGamal signature verification__ protocol to verify a signed message $m$ with signature $(r,s)$ where the public key of the signer is $y$:
1. Verify that $0 < r < p$ and $0 < s < p − 1$.
2. Verify that $g^{H(m)} \equiv y^r r^s \pmod p$.

In [37]:
# Function to read a proxy signature from a file
def read_signature_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        r = int(lines[0].split(': ')[1])
        s = int(lines[1].split(': ')[1])
        K = int(lines[2].split(': ')[1])
        return r, s, K

# Read the proxy signature from a file proxy_signature.txt
signature_filename = 'proxy_signature.txt'
r, s, bigk = read_signature_from_file(signature_filename)
print('Proxy signature read from file:', (r, s, bigk))

message_filename = 'message.txt'
m = read_message_from_file(message_filename)
print('Message read from file:', m)

print('r =',r,', s =',s,', p =',p)
check1 = False
check2 = False
# Verification Check 1
if (0<r) & (r<p):
    if (0<s) & (s<(p-1)):
        check1 = True
        print('Verification Check 1: Passed')
    else:
        print('Verification Check 1: Failed')
else:
    print('Verification Check 1: Failed')

Proxy signature read from file: (2957268667464979456, 1135264699249436940, 1703719962413632553)
message: Ado mama Anjula
Message read from file: 92936444892425296126897496249657084158512872626459328642006554000005634312468
r = 2957268667464979456 , s = 1135264699249436940 , p = 5032252537865353033
Verification Check 1: Passed


In [38]:
# Assuming rhs is the proxy public key and lhs is defined as follows
y = (v * pow(bigk, bigk, p)) % p  # Proxy public key y = VK^K mod p

# Compute LHS = g^m % p efficiently
lhs = pow(g, m, p)

# Compute RHS = (y^r * r^s) % p efficiently
rhs = (pow(y, r, p) * pow(r, s, p)) % p

# Print results
print('LHS:', lhs)
print('RHS:', rhs)

# Verify if LHS equals RHS
if lhs == rhs:
    check2 = True
    print('Verification Check 2: Passed')
else:
    check2 = False
    print('Verification Check 2: Failed')


LHS: 3756076729166452990
RHS: 3756076729166452990
Verification Check 2: Passed


In [39]:
if(check1 & check2):
    print('Proxy signature verification: Passed')
else:
    print('Proxy signature verification: Failed')

Proxy signature verification: Passed


In [73]:
import random
genrated_number = random.getrandbits(1024)
print(genrated_number)

97773333501964145945851685283748420244849612380276164018218382747614908598279153173622708263024100976032803749294853176801279578526308087119884174093528963736076497737699169814652156047617941147626907362210156548586211550751163850079356229550574339938448978911498408934087353682117698189092856543278608851725


In [82]:
# modulus addition

mod_512 = 2**512
mod_1024 = 2**1024
mod_2048 = 2**2048

value = ((-87598250036046328361573979505897579787456709099174749148423254841095346166072716324373693401289806040349925933362124410478422643300338463347490089195912682110518980566451036360852877358396540284568447404069928935265327993407370576884662731932897796849068204195588940852651911426296135682164641491366577360037
         * -21719959093543122193271597345913092765662363758920770284300105782111269136091634723077329996229524739293188973726960630197805213008905755855383760381155029228488818252788461905700483460905197645298493264714083598738790014487001568691651326578867087973323242769874902344568522847443006422922857740695323511933) 
        % mod_1024)
print(value)

152642452731986236266917600130708390579028167712776439390699028344909777667204382863740435807987428154387474465444171074509230880947785368936452777958972785382354187915618459205899645333281563009066484139615965359723310359933667317792118326106773678447759884378243046264543382049899602593075709467366738045073
